In [3]:
import os
import sys

#  Set the working directory to the folder containing the top-level ultralytics package
os.chdir("/workspace")  # change to your workspace root where ultralytics folder exists
print("Current working directory:", os.getcwd())


Current working directory: /workspace


In [3]:
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics


Cloning into 'ultralytics'...
remote: Enumerating objects: 67800, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 67800 (delta 61), reused 30 (delta 21), pack-reused 67677 (from 2)
Receiving objects: 100% (67800/67800), 36.01 MiB | 8.35 MiB/s, done.
Resolving deltas: 100% (50671/50671), done.
Updating files: 100% (778/778), done.
/workspace/ultralytics


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import os
import sys

#  Set the working directory to the folder containing the top-level ultralytics package
print("Current working directory:", os.getcwd())


Current working directory: /workspace


In [5]:
#  Add the top-level ultralytics folder to Python path
os.chdir("/workspace/ultralytics")
print("Current working directory:", os.getcwd())

Current working directory: /workspace/ultralytics


In [33]:
from ultralytics import YOLO

# -------------------------------
# Load YAML (with original SPPF)
# -------------------------------
model = YOLO('yolov8.yaml')
print("Original model loaded ")


WARNING ⚠️ no model scale passed. Assuming scale='n'.
Original model loaded 


In [5]:
import ultralytics
print(ultralytics.__file__)


/workspace/ultralytics/ultralytics/__init__.py


In [34]:
from ultralytics.models.yolo.model import YOLO

# Load your YOLOv8 model (with CBAM if added)
model = YOLO("ultralytics/cfg/models/v8/yolov8.yaml")

# Print the full architecture
print(model.model)   # model.model contains the actual PyTorch nn.Module


WARNING ⚠️ no model scale passed. Assuming scale='n'.
DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stat

In [35]:
print("Current working directory:", os.getcwd())


Current working directory: /workspace/ultralytics


In [27]:
import torch
from ultralytics import YOLO

# --- Load model ---
model = YOLO("yolov8.yaml")  # your custom model
model.eval()  # evaluation mode

# --- Create dummy input ---
dummy_img = torch.randn(1, 3, 640, 640)  # batch_size=1, 3 channels, 640x640

# --- Forward pass through the model ---
with torch.no_grad():
    preds = model.model(dummy_img)  # list of outputs from YOLO layers

# --- Pick one output and compute a dummy EIoU loss ---
from ultralytics.utils.loss import BboxLoss
from ultralytics.utils.metrics import bbox_iou

# fake target boxes (x1,y1,x2,y2)
target_boxes = torch.tensor([[100, 100, 200, 200]], dtype=torch.float32)
pred_boxes = torch.tensor([[110, 110, 210, 210]], dtype=torch.float32)

bbox_loss_fn = BboxLoss()
# Mask & dummy target scores
fg_mask = torch.tensor([True])
target_scores = torch.tensor([[1.0]])
target_scores_sum = torch.tensor(1.0)

loss_iou, loss_dfl = bbox_loss_fn(
    pred_dist=torch.zeros(1, 16, 4),      # dummy DFL input
    pred_bboxes=pred_boxes,
    anchor_points=torch.zeros(1, 2),      # dummy anchors
    target_bboxes=target_boxes,
    target_scores=target_scores,
    target_scores_sum=target_scores_sum,
    fg_mask=fg_mask
)

print(f"[Sanity Check] loss_iou = {loss_iou.item()}, loss_dfl = {loss_dfl.item()}")


WARNING ⚠️ no model scale passed. Assuming scale='n'.
[Sanity Check] loss_iou = 0.3275921940803528, loss_dfl = 2.7725887298583984


In [ ]:
from ultralytics import YOLO
import time
import torch

# === Device setup ===
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# === Load default YOLOv8 architecture ===
model = YOLO("yolov8.yaml").to(device)  # Use default YAML, not custom CBAM YAML

# === Set box loss to EIoU only ===
model.box_loss_type = "eiou"  # Enable EIoU
# Classification and objectness remain default BCE
model.cls_loss_type = "bce"  # placeholder (doesn't impact)
model.obj_loss_type = "bce"  # placeholder (doesn't impact)

# === Enable EMA ===
model.ema = True

# === Training parameters ===
dataset_yaml = "/workspace/datasets/KITTI/kitti.yml"  # Custom KITTI dataset YAML
run_name = "yolov8n_eiou_base_hypers_pat40"  # Run name for saving logs and weights
epochs = 2
imgsz = 1280
batch_size = 32
workers = 2
cache_images = "disk"
amp = True
save_interval = 50
patience = 40

# === EMA status ===
print("\n=== EMA STATUS ===")
print(f"EMA enabled: {model.ema}")

# === Detect layer info ===
print("\n=== DETECT LAYER INFO ===")
detect_layer = model.model.model[-1]
print(f"  Classes (nc): {detect_layer.nc}")
try:
    print(f"  Strides: {detect_layer.stride}")
    print(f"  Number of detection heads: {len(detect_layer.stride)}")
except Exception as e:
    print(f"  Could not read strides/heads: {e}")

# === Model summary ===
print("\n=== MODEL SUMMARY ===")
print(model.model)

# === Loss type verification ===
print("\n=== LOSS TYPE VERIFICATION ===")
print(f"Box loss type: {model.box_loss_type}")
print(f"Class loss type: {model.cls_loss_type}")
print(f"Object loss type: {model.obj_loss_type}")

# === Start full training ===
print("\n=== STARTING FULL TRAINING ===")
start_time = time.time()

results = model.train(
    data=dataset_yaml,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    workers=workers,
    device=device,
    cache=cache_images,
    name=run_name,
    save=True,
    amp=amp,
    patience=patience,
    save_period=save_interval,
    pretrained="yolov8n.pt",  # Use COCO pretrained weights
    optimizer="AdamW",
    lr0=0.001,
    weight_decay=1e-4,
    cos_lr=True,
    lrf=0.01,
    augment=True
)

total_time = time.time() - start_time
print(f"\nTraining completed in {total_time/3600:.2f} hours")



=== EMA STATUS ===
EMA enabled: True

=== DETECT LAYER INFO ===
  Classes (nc): 7
  Strides: tensor([ 8., 16., 32.], device='cuda:0')
  Number of detection heads: 3

=== MODEL SUMMARY ===
DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32,

In [ ]:
from ultralytics import YOLO
import time
import torch

# === Device setup ===
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# === Load default YOLOv8 architecture ===
model = YOLO("yolov8.yaml").to(device)  # Use default YAML, not custom CBAM YAML

# === Set box loss to EIoU only ===
model.box_loss_type = "eiou"  # Enable EIoU
# Classification and objectness remain default BCE
model.cls_loss_type = "bce"  # placeholder (doesn't impact)
model.obj_loss_type = "bce"  # placeholder (doesn't impact)

# === Enable EMA ===
model.ema = True

# === Training parameters ===
dataset_yaml = "/workspace/datasets/KITTI/kitti.yml"  # Custom KITTI dataset YAML
run_name = "yolov8n_eiou_base_hypers_pat40"  # Run name for saving logs and weights
epochs = 250
imgsz = 1280
batch_size = 32
workers = 2
cache_images = "disk"
amp = True
save_interval = 50
patience = 40

# === EMA status ===
print("\n=== EMA STATUS ===")
print(f"EMA enabled: {model.ema}")

# === Detect layer info ===
print("\n=== DETECT LAYER INFO ===")
detect_layer = model.model.model[-1]
print(f"  Classes (nc): {detect_layer.nc}")
try:
    print(f"  Strides: {detect_layer.stride}")
    print(f"  Number of detection heads: {len(detect_layer.stride)}")
except Exception as e:
    print(f"  Could not read strides/heads: {e}")

# === Model summary ===
print("\n=== MODEL SUMMARY ===")
print(model.model)

# === Loss type verification ===
print("\n=== LOSS TYPE VERIFICATION ===")
print(f"Box loss type: {model.box_loss_type}")
print(f"Class loss type: {model.cls_loss_type}")
print(f"Object loss type: {model.obj_loss_type}")

# === Start full training ===
print("\n=== STARTING FULL TRAINING ===")
start_time = time.time()

results = model.train(
    data=dataset_yaml,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    workers=workers,
    device=device,
    cache=cache_images,
    name=run_name,
    save=True,
    amp=amp,
    patience=patience,
    save_period=save_interval,
    pretrained="yolov8n.pt",  # Use COCO pretrained weights
    optimizer="AdamW",
    lr0=0.001,
    weight_decay=1e-4,
    cos_lr=True,
    lrf=0.01,
    augment=True
)

total_time = time.time() - start_time
print(f"\nTraining completed in {total_time/3600:.2f} hours")



=== EMA STATUS ===
EMA enabled: True

=== DETECT LAYER INFO ===
  Classes (nc): 7
  Strides: tensor([ 8., 16., 32.], device='cuda:0')
  Number of detection heads: 3

=== MODEL SUMMARY ===
DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32,

In [12]:
import sys
from pathlib import Path
import torch
import time
from ultralytics import YOLO  # Use YOLOv8's interface

weights = "runs/detect/yolov8n_eiou_base_hypers_pat402/weights/best.pt"
dataset_yaml = "/workspace/datasets/KITTI/kitti.yml"
results_dir = "/workspace/yolov8n_eiou_base_hypers_pat402_val"

# ------------------- Device -------------------
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# ------------------- Load model -------------------
model = YOLO(weights)  # Load the model using YOLOv8
params = sum(p.numel() for p in model.model.parameters())
params_mb = params * 4 / (1024**2)  # float32 -> 4 bytes
model_size_mb = Path(weights).stat().st_size / 1024**2  # file size in MB

print(f"Model size (file): {model_size_mb:.2f} MB")
print(f"Model parameters: {params_mb:.2f} MB")

# ------------------- FPS Measurement -------------------
dummy_input = torch.randn(1, 3, 1280, 1280).to(device) / 255.0  # Normalize the dummy input
# Warm-up
for _ in range(5):
    _ = model(dummy_input)  # Use the model directly for inference

n_runs = 50
start_time = time.time()
for _ in range(n_runs):
    _ = model(dummy_input)  # Use the model directly for inference
end_time = time.time()

fps = n_runs / (end_time - start_time)
print(f"Inference FPS (1280x1280): {fps:.2f}")

# ------------------- Run Validation -------------------
print("\nRunning YOLOv8 validation...")
start_val_time = time.time()
model.val(
    data=dataset_yaml,  # Validation dataset
    imgsz=1280,          # Image size
    batch=64,            # Corrected batch size argument
    device=device,       # Device
    project=results_dir, # Results directory
    name="eval_metrics", # Name for saved results
    save_json=True,      # Save results as JSON
    exist_ok=True,       # Overwrite if results directory exists
    verbose=True         # Show verbose output
)
end_val_time = time.time()

val_time = end_val_time - start_val_time
print(f"Validation time: {val_time:.2f} seconds")
print(f"Validation results saved to {results_dir}")


Model size (file): 6.00 MB
Model parameters: 11.49 MB

0: 1280x1280 (no detections), 5.3ms
Speed: 0.0ms preprocess, 5.3ms inference, 1.4ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 1.2ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.6ms
Speed: 0.0ms preprocess, 5.6ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)



In [6]:
import shutil

# Specify the folder path and the destination zip file path
folder_path = 'runs/detect/yolov8n_eiou_base_hypers_pat40'
zip_file_path = 'runs/detect/yolov8n_eiou_base_hypers_pat40.zip'

# Create a zip file from the folder
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

'/workspace/ultralytics/runs/detect/yolov8n_eiou_base_hypers_pat40.zip'

In [2]:
import shutil

# Specify the folder path and the destination zip file path
folder_path = '/workspace/yolov8n_eiou_base_hypers_pat40_val'
zip_file_path = '/workspace/yolov8n_eiou_base_hypers_pat40_val.zip'

# Create a zip file from the folder
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

'/workspace/yolov8n_eiou_base_hypers_pat40_val.zip'